# Preprocessing des metadata UKB

In [ ]:
import pandas as pd
from fancyimpute import KNN
import numpy as np
from sklearn.preprocessing import LabelEncoder
import os

os.chdir("/home/tbarba/projects/MultiModalBrainSurvival")

META = "data/metadata/UKB/whole/UKB_metadata_full.gz"
LIST_VAR = "data/metadata/UKB/whole/UKB_variables_all_new.xlsx"
MR_DATASET = [int(c) for c in os.listdir("data/MR/UKBIOBANK")]

# IMPORT AND RENAMING VAR COLUMNS
df = pd.read_csv(META)
df = df[df['eid'].isin(MR_DATASET)]
listvar = pd.read_excel(LIST_VAR)
listvar = listvar[listvar['keep_model']]


keep_var =  set(listvar["var"]) & set(df.columns)
df = df[["eid"] + list(keep_var)]


# DUMP DATA WITH > 20% MISSINGNESS
to_dump = []
to_keep = []
listvar = listvar[listvar['var'].isin(keep_var)]

for var in df.columns:
    missingness = df[var].isna().sum(axis=0) / df.shape[0]
    if missingness > .20:
        to_dump.append(var)
    else:
        to_keep.append(var)

final_df = df[to_keep]
final_df = final_df.set_index("eid")


# MISSING DATA IMPUTATION
labelencoder = LabelEncoder()
final_df = final_df.apply(lambda series: pd.Series(
    labelencoder.fit_transform(series[series.notnull()]),
    index=series[series.notnull()].index
))

imputer = KNN(120)
print("missing data imputation...")
final_df = pd.DataFrame(np.round(imputer.fit_transform(final_df)), index=final_df.index, columns=final_df.columns)


# EXPORTING METADATA FILES
final_df.to_csv("data/metadata/UKB/MR_cohort/imputed.csv", index=True)
listvar.to_csv("data/metadata/UKB/MR_cohort/variable_list.csv", index=False)


/tmp/ipykernel_32779/3823462232.py:14: DtypeWarning: Columns (97,115,117,140,166,205,206,213,214,217,218,219,220,221,222,227,228,229,230,235,236,237,238) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(META)


In [22]:
import pandas as pd
from fancyimpute import KNN
import numpy as np
from sklearn.preprocessing import LabelEncoder
import os

os.chdir("/home/tbarba/projects/MultiModalBrainSurvival")

META = "data/metadata/UKB/whole/UKB_metadata_full.gz"
LIST_VAR = "data/metadata/UKB/whole/UKB_variables_all_new.xlsx"
MR_DATASET = [int(c) for c in os.listdir("data/MR/UKBIOBANK")]

# IMPORT AND RENAMING VAR COLUMNS
df = pd.read_csv(META)
df = df[df['eid'].isin(MR_DATASET)]
listvar = pd.read_excel(LIST_VAR)
listvar = listvar[listvar['keep_model']]


/tmp/ipykernel_32779/3932562098.py:14: DtypeWarning: Columns (97,115,117,140,166,205,206,213,214,217,218,219,220,221,222,227,228,229,230,235,236,237,238) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(META)


In [27]:
variables = ["Alcohol_intake_frequency", "stroke_delay"]
a = listvar[listvar['var'].isin( variables)]
results = {"Alcohol_intake_frequency":10, "stroke_delay":20}

for i in a['var']:
    a.loc[a['var'] == i, "res"] = fakeres[i]

In [33]:
import pandas as pd
multi = pd.read_csv("/home/tbarba/projects/MultiModalBrainSurvival/outputs/UNet/pretraining/multi_summary.csv")
uni = pd.read_csv("/home/tbarba/projects/MultiModalBrainSurvival/outputs/UNet/pretraining/univ_summary.csv")

uni['var'] = uni['newname'] 
uni = uni.drop("newname", axis=1).set_index(["model", "var"]).sort_index()
multi = multi.set_index(["model", "var"]).sort_index()

In [37]:

uni

code               group  \
model             var                                                   
UNet_5b_4f_UKfull age                       21022              global   
                  alcohol_frequency          1558             alcohol   
                  alcohol_status            20117             alcohol   
                  anxiety                    1980                 psy   
                  arterial_stiffness_index  21021  arterial_stiffness   
...                                           ...                 ...   
UNet_6b_8f_UKfull wk_fortifwine_intake       1608             alcohol   
                  wk_redwine_intake          1568             alcohol   
                  wk_spirits_intake          1598             alcohol   
                  wk_whitewine_intale        1578             alcohol   
                  worry                      2000                 psy   

                                                      task  comment   dump  \
model             var                                                        
UNet_5b_4f_UKfull age                           regression      NaN  False   
                  alcohol_frequency             regression  ordered  False   
                  alcohol_status            classification      NaN  False   
                  anxiety                   classification      NaN  False   
                  arterial_stiffness_index      regression      NaN  False   
...                                                    ...      ...    ...   
UNet_6b_8f_UKfull wk_fortifwine_intake          regression      NaN  False   
                  wk_redwine_intake             regression      NaN  False   
                  wk_spirits_intake             regression      NaN  False   
                  wk_whitewine_intale           regression      NaN  False   
                  worry                     classification      NaN  False   

                                            proportion_sig  
model             var                                       
UNet_5b_4f_UKfull age                             0.407617  
                  alcohol_frequency               0.162012  
                  alcohol_status                  0.015430  
                  anxiety                         0.082324  
                  arterial_stiffness_index        0.172852  
...                                                    ...  
UNet_6b_8f_UKfull wk_fortifwine_intake            0.049072  
                  wk_redwine_intake               0.062012  
                  wk_spirits_intake               0.000000  
                  wk_whitewine_intale             0.783691  
                  worry                           0.150146  

[388 rows x 6 columns]

In [40]:
multi2 = pd.merge(uni, multi, left_index=True, right_index=True)
multi2 = multi2[["code","group","task","comment","dump","performance"]]

In [42]:
multi2.to_csv("/home/tbarba/projects/MultiModalBrainSurvival/outputs/UNet/pretraining/multi_summary2.csv", index=True)